<a href="https://colab.research.google.com/github/rrsartneoai/Awesome-Multimodal-Large-Language-Models/blob/main/finetune_rafal_stepkowski.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

  Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (39.9 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.1
    Uninstalling pyarrow-14.0.1:
      Successfully uninstalled pyarrow-14.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
!pip install pyarrow==14.0.1

  Using cached pyarrow-14.0.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
Using cached pyarrow-14.0.1-cp310-cp310-manylinux_2_28_x86_64.whl (38.0 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.21.0 requires pyarrow>=15.0.0, but you have pyarrow 14.0.1 which is incompatible.


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments

from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset('text', data_files={'train': 'train.txt', 'test': 'test.txt'})

In [ ]:
# Inicjalizacja tokenizer'a i modelu

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

In [ ]:
# Tokenizacja danych

def tokenize_function(examples):
    # Indent the return statement to be part of the function
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
# Parametry fine-tuningu

training_args = TrainingArguments(
output_dir='./results',
num_train_epochs=3, # Liczba epok
per_device_train_batch_size=16, # Batch size
per_device_eval_batch_size=16,
learning_rate=2e-5, # Learning rate
eval_strategy="epoch", # updated from evaulation to eval - rafal
logging_dir='./logs',
)

In [ ]:
# Inicjalizacja i uruchomienie fine-tuningu - Rafal
import numpy as np
from transformers import Trainer, TrainingArguments
from datasets import load_metric

# Load the metric
metric = load_metric("accuracy")  # Or any other suitable metric

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].add_column('labels', [0]*len(tokenized_datasets["train"])),  # Add dummy labels for demonstration
    eval_dataset=tokenized_datasets["test"].add_column('labels', [0]*len(tokenized_datasets["test"])),  # Add dummy labels for demonstration
    compute_metrics=compute_metrics,
)

# Start the training process
trainer.train()